# Geometry-based eddy detection and tracking algorithm

1. Identify eddies in Gulf of Mexico during sample time period
2. Track the eddies during the time period  

Authors: Martin Jang, Phillip J. Wolfram

Version: 2.0

## Setup

In [ ]:
import numpy as np
from netCDF4 import Dataset, date2num
import matplotlib.pyplot as plt
from datetime import datetime
#from detect_eddy import *
#from utils import *
import pickle
#from Vortex import Doublelist
import os
import sys

import r2_detect_eddy

## Detection
Zonal and meridional surface velocity field is to be examined for eddy identification. The detection scheme is explained in more details in "detect_center.py" For this demo, one month period (January 2012) of HYCOM data is used.    

In [ ]:
year = 2012

# Load latitude and longitude in Gulf of Mexico
Y, X = loadLatLon('GOM')

# Load num_time of corresponding year
# num_time is datetime in hours since 2000/01/01 00:00:00
with Dataset('gom_reanalysis.nc', 'r') as source:
    num_time = source['time/' + str(year)][:]
    units = source['time/' + str(year)].units
    calendar = source['time/' + str(year)].calendar

# Find num_time of first and last day of sample time period
num_year_start = date2num(datetime(year, 1, 1, 0, 0,), units, calendar)
num_year_end = date2num(datetime(year, 2, 1, 0, 0,), units, calendar)  

# Calculate number of days in each year
number_of_days = np.int((num_year_end - num_year_start) / 24)

# Initialize variables
detection = np.ma.zeros((number_of_days, 325, 430))
divergence = np.ma.zeros((number_of_days, 325, 430))
vorticity = np.ma.zeros((number_of_days, 325, 430))
okuboWeiss = np.ma.zeros((number_of_days, 325, 430))
uVel = np.ma.zeros((number_of_days, 325, 430))
vVel = np.ma.zeros((number_of_days, 325, 430))

# Initialize first day of each year
day_start = num_year_start

# Compute dx and dy
dx, dy = dxdy('GOM')

day_count = 0
while (day_start < num_year_end):
    day_end = day_start + 24
    time = np.where((day_start <= num_time) & (num_time < day_end))[0]
    day_start = day_end

    # Load velocity for each day of year
    u, v = loadVelocity_GOM(year, time)        
    
    # Detect eddy centers
    detection_result = detect_center(u, v)
    
    # Calculate divergence, vorticity, and Okuno-Weiss parameter
    div, vort, ow = dvo(u, v, dx, dy)    
    
    # Store results
    detection[day_count, :, :] = detection_result
    divergence[day_count, :, :] = div
    vorticity[day_count, :, :] = vort
    okuboWeiss[day_count, :, :] = ow
    uVel[day_count, :, :] = u
    vVel[day_count, :, :] = v
    
    day_count += 1
    print "Searching ({}/{})".format(day_count, number_of_days)

# plot detection result

In [ ]:
day = 1
plotDetection(detection[day, :, :], vorticity[day, :, :], okuboWeiss[day, :, :], uVel[day, :, :], vVel[day, :, :], Y, X)

# new Williamas R2 algorithm

 1. create eddy mask region via Okubo Weiss thresholding (vs -0.2 as default)
 2. label continguous regions on the mask as distinct features
 3. evaluate these features building 1D coordinates out from isosurfaces (use marching-approach)
 4. evaluate the feature area vs isosurface based on r2 condition to find eddy
 5. search remainder of feature and rest of features to find all eddies to produce a labeled eddy mask for the domain
 6. assume eddy "center" is at region of minimum Okubo Weiss value in each eddy mask region

In [ ]:
import r2_detect_eddy
reload(r2_detect_eddy)
# identify distinct regions of possible eddies
mask, nfeatures = r2_detect_eddy.ow_eddy_labeled_mask(ow, mineddycells=100)

In [ ]:
ow = okuboWeiss[0,:,:]

mask,_ = r2_detect_eddy.ow_eddy_labeled_mask(ow, mineddycells=100)
maskfull,_ = r2_detect_eddy.ow_eddy_labeled_mask(ow)
x, y = np.meshgrid(X, Y)
plt.pcolor(x,y,mask)
plt.clim(25,27)
plt.colorbar()

In [ ]:
x, y = np.meshgrid(X, Y)
plt.pcolor(x,y,mask)
plt.colorbar()
plt.figure()
plt.pcolor(x,y,maskfull)
plt.colorbar()

In [ ]:
owtest = ow.copy()
owtest[mask != 28] = 0
plt.pcolor(x,y,owtest)
plt.colorbar()

In [ ]:
reload(r2_detect_eddy)
fnum = 28
feature = (mask==fnum)
for r2 in [0.5, 0.75, 0.9, 0.95]:
    foundeddy, eddyfeature = r2_detect_eddy.find_eddy(feature, ow, dx*dy, r2cond=r2)
    plt.figure(figsize=(5,10))
    plt.subplot(2,1,1)
    plt.pcolor(eddyfeature)
    plt.subplot(2,1,2)
    plt.pcolor(feature)
    r2_detect_eddy.r2check(ow[eddyfeature], (dx*dy)[eddyfeature], plot=True, minr2points=-1)

In [ ]:
reload(r2_detect_eddy)
alleddies = r2_detect_eddy.find_all_eddies(ow, dx*dy, mineddycells=100, minr2points=30, r2cond=0.75)
plt.pcolor(x, y, mask)
plt.colorbar()
plt.figure()
plt.pcolor(x, y, alleddies)
plt.colorbar()

In [ ]:
plt.pcolor((alleddies!=4))
reload(r2_detect_eddy)
r2_detect_eddy.r2check(ow[alleddies==4], (dx*dy)[alleddies==4], plot=True, minr2points=-1)

In [ ]:
reload(r2_detect_eddy)
alleddies = r2_detect_eddy.find_all_eddies(ow, dx*dy, mineddycells=100, minr2points=30)
eddycenters = r2_detect_eddy.eddy_centers(alleddies, ow)
plt.pcolor(eddycenters)
plt.colorbar()
plt.figure()
plt.pcolor(alleddies)

In [ ]:
day=0
ow = okuboWeiss[day,:,:]

import numpy.ma as ma
import r2_detect_eddy
reload(r2_detect_eddy)

for r2 in [0.5, 0.75, 0.9, 0.95]:
    
    # compute eddies
    alleddies = r2_detect_eddy.find_all_eddies(ow, dx*dy, mineddycells=100, r2cond=r2, minr2points=30)
    eddycenters = r2_detect_eddy.eddy_centers(alleddies, ow)
    eddycenters = ma.array(data=eddycenters, mask = ow.mask)

    # plot eddies
    plt.figure()
    plotDetection(eddycenters, vorticity[day, :, :], okuboWeiss[day, :, :]*(alleddies > 0), uVel[day, :, :], vVel[day, :, :], Y, X)
    plt.title('Williams method with r$^2$={:.2f}'.format(r2))
    
plt.figure()
plotDetection(detection[day, :, :], vorticity[day, :, :], okuboWeiss[day, :, :], uVel[day, :, :], vVel[day, :, :], Y, X)
plt.title('Nencioli method')

## plot
Shows center of detected eddies with velocity field and vorticity dominant regions.

In [ ]:
import numpy.ma as ma
import r2_detect_eddy
reload(r2_detect_eddy)

for day in np.arange(31):    # choose between 1 ~ 31
    for r2 in [0.5, 0.75, 0.9, 0.95]:
        
        plt.close()

        # compute eddies
        ow = okuboWeiss[day, :, :]
        alleddies = r2_detect_eddy.find_all_eddies(ow, dx*dy, 
                                                   mineddycells=100, r2cond=r2, minr2points=30)
        eddycenters = r2_detect_eddy.eddy_centers(alleddies, ow)
        eddycenters = ma.array(data=eddycenters, mask=okuboWeiss[0, :, :].mask)

        # plot eddies
        plt.figure()
        plotDetection(eddycenters, vorticity[day, :, :], okuboWeiss[day, :, :]*(alleddies > 0), uVel[day, :, :], vVel[day, :, :], Y, X)
        plt.title('Williams method with r$^2$={:.2f}'.format(r2))
        plt.title('day {:2d}'.format(day+1))
        plt.xlim(-98, -82)
        plt.ylim(18, 32)
        plt.savefig('{:.2f}WilliamsDay{:02d}.png'.format(r2,day+1))
    
    plt.figure()
    plotDetection(detection[day, :, :], vorticity[day, :, :], okuboWeiss[day, :, :], uVel[day, :, :], vVel[day, :, :], Y, X)
    plt.title('Nencioli method')
    plt.title('day {:2d}'.format(day+1))
    plt.xlim(-98, -82)
    plt.ylim(18, 32)
    plt.savefig('NenioliDay{:02d}.png'.format(day+1))

# make a movie

In [ ]:
# make a movie
from subprocess import call
def make_movie(moviename):
    
    import os
    os.remove(moviename + '.mp4')
    
    args = ['ffmpeg', 
            '-framerate', '3',
            '-pattern_type', 'glob', 
            '-i', moviename + '*.png',
            '-c:v', 'libx264', 
            '-r', '30',
            '-profile:v', 'high',
            '-crf', '20', 
            '-pix_fmt', 'yuv420p', 
            moviename +  '.mp4']
    call(args)
    
williamsnames = ['{:.2f}WilliamsDay'.format(anum) for anum in [0.5, 0.75, 0.9, 0.95]]
    
for aname in ['NenioliDay'] + williamsnames:
    print aname
    make_movie(aname)

## Tracking

In [ ]:
eddy_centers = detection.copy()
time_idx, lat_idx, lon_idx = eddy_centers.shape    

# Initialize eddy counter
eddy_counter = 0
# Set searching window width
searching_width = 21    # odd number; approx. 30 miles = 48km for GOM

for t in np.arange(time_idx):

    # Cyclonic eddy    
    for i, j in zip(*np.where(eddy_centers[t, :, :] == 1)):

        # Create a new eddy object
        eddy_counter += 1
        globals()["eddy_" + str(eddy_counter)] = Doublelist()

        # convert t to num_time
        num_time_t = date2num(datetime(year, 1, 1,), units, calendar) + t * 24
        
        # record num_time, lat, lon, and mode
        globals()["eddy_" + str(eddy_counter)].append(num_time_t, i, j, 1)

        # Place a check mark at the location by change 1 to NaN
        eddy_centers[t, i, j] = np.NaN

        # While switch is on, search and append to eddy track list if search is successful
        switch = True
        
        # Copy the time and location of current eddy
        search_day = t.copy(); search_lat_idx = i.copy(); search_lon_idx = j.copy()
        while switch:
            # Look for an eddy or eddies inside the searching window
            search_day, search_lat_idx, search_lon_idx = search(search_day, search_lat_idx, search_lon_idx, 1, searching_width, eddy_centers, time_idx)             
            if search_day != None:
                # convert search_day to num_time
                num_time_search_day = date2num(datetime(year, 1, 1,), units, calendar) + search_day * 24
                
                # record num_time, lat, lon, and mode
                globals()["eddy_" + str(eddy_counter)].append(num_time_search_day, search_lat_idx, search_lon_idx, 1)

                # Place a check mark at the location by change 1 to NaN
                eddy_centers[search_day, search_lat_idx, search_lon_idx] = np.NaN
            else:
                switch = False

    # Anti-cyclonic eddy    
    for i, j in zip(*np.where(eddy_centers[t, :, :] == -1)):

        # Create new eddys
        eddy_counter += 1
        globals()["eddy_" + str(eddy_counter)] = Doublelist()

        # convert t to num_time
        num_time_t = date2num(datetime(year, 1, 1,), units, calendar) + t * 24
        
        # record num_time, lat, lon, and mode
        globals()["eddy_" + str(eddy_counter)].append(num_time_t, i, j, -1)

        # Place a check mark at the location by change 1 to NaN
        eddy_centers[t, i, j] = np.NaN

        # While switch is on, search and append to eddy track list if search is successful
        switch = True
        search_day = t.copy(); search_lat_idx = i.copy(); search_lon_idx = j.copy()
        while switch:
            # Look for an eddy or eddies inside the searching window
            search_day, search_lat_idx, search_lon_idx = search(search_day, search_lat_idx, search_lon_idx, -1, searching_width, eddy_centers, time_idx) 
            if search_day != None:

                # convert search_day to num_time
                num_time_search_day = date2num(datetime(year, 1, 1,), units, calendar) + search_day * 24
                
                # record num_time, lat, lon, and mode
                globals()["eddy_" + str(eddy_counter)].append(num_time_search_day, search_lat_idx, search_lon_idx, -1)

                # Place a check mark at the location by change 1 to NaN
                eddy_centers[search_day, search_lat_idx, search_lon_idx] = np.NaN
            else:
                switch = False

print "Total number of eddies detected: {}".format(eddy_counter)

## View eddy track sample

The total number of eddies detected is equal to the number of eddy objects created. So, for example when 100 eddies are detected, 100 eddy objects (eddy_1, eddy_2, ...., eddy_100) would have been created and each eddy can be access by methods such as show(), lifetime(), mode(), and speed(). 

In [ ]:
eddy_1.show()
print(eddy_1.lifetime())
print(eddy_1.mode())
print(eddy_1.speed())

In [ ]:
def find_local_mins(A, Astart=-1.0, neigh=3, maxevalpoints=30):
    nx, ny = A.shape
    localmins = np.maximum(nx,ny)*np.ones((2,maxevalpoints),dtype='i')
    imin=0
    for j in np.arange(neigh, ny-neigh):
        for i in np.arange(neigh, nx-neigh):
            Aminneigh = np.min(A[i-neigh:i+neigh,j-neigh:j+neigh])
            if A[i,j] < Astart and A[i,j] == Aminneigh:
                localmins[:,imin] = (i,j)
                imin += 1
                if imin == maxevalpoints:
                    return localmins
    
    # remove hanging nans
    localmins = localmins[:,:imin]
    return localmins
#mins = find_local_mins(okuboWeiss[0,:,:], neigh=10, 
#                       maxevalpoints=np.prod(okuboWeiss[0,:,:].shape))